Analysis Strategy
### Select tables, columns and rows needed.

- Select the required fields from message, user_group_user, and channel tables and merge them.
- Filter the threads that are non-standalone based on the thread_ts, subtype is null and ‘thread_broadcast’, channel not null, not memes, not social, not networking, not alumni, not announcements.

### Determine the reply time per message of users between two teams.

- Create a lag_ts and lag_user_group_id within each thread_ts.
- Calculate the reply time by subtracting lag_ts from ts within thread_ts.
- Filter rows for user_group_id ≠ lag_user_group_id (this ensures that the message is exchanged between teams)
- Create a field that concats the user_id_group and lag_user_id_group.
- Calculate the average time that each team takes to respond to the message from the other team.
- Calculate the total number of threads initiated between the two teams to know how frequently they communicate.
- Create a table containing Sender, replier, average_reply_time, total number of threads initiated between the teams.

Assumption:
* All users in the message can be a sender and replier.

Communication Analysis Insights

1. Intra-team Communication

* Teams communicate more frequently internally than between teams
* Leadership shows highest internal communication frequency
* Security team demonstrates lowest internal communication

2. Leadership Interaction

* After internal communications, interaction with leadership is most frequent across all teams

3. Cross-team Collaboration

* Communication between the three teams (Engineering, Data, Security) remains weak overall

4. Internal Response Times

* Teams respond faster to their own members
* Security team has quickest internal response time (~8 minutes)
* Engineering team has slowest internal response time (~28 minutes)

5. Leadership Response Dynamics

* Leadership responds promptly to all teams (≤10 minutes average)
* Engineering and Data teams respond quickly to leadership (<10 minutes)
* Security team has significant delay responding to leadership (>90 minutes)

6. Inter-team Responsiveness

* Engineering team shows poorest cross-team response times (>20 hours to both Data and Security)
* Security team has delayed responses to other teams (>10 hours)
* Data team demonstrates better overall responsiveness to all teams


In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# create a connection string
conn_string = (
    "postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
).format(
    username="***",
    password="***",
    host="*****",
    port="****",
    database="******"
)
# Create engine
engine = create_engine(conn_string)

In [32]:
# Extract the desired tables, columns and rows from the database.
query3 = """
SELECT u.user_group_id AS sender_team,
       m.thread_ts,
       m.user_id AS sender_user_id,
       c.name,
       to_timestamp(CAST(m.ts AS DOUBLE PRECISION)) AS ts_new,
       LEAD(u.user_id) OVER(PARTITION BY thread_ts ORDER BY m.ts) AS replier_user_id,
       LEAD(u.user_group_id) OVER(PARTITION BY thread_ts ORDER BY m.ts) AS replier_team,
       LEAD(to_timestamp(CAST(m.ts AS DOUBLE PRECISION))) OVER(PARTITION BY thread_ts ORDER BY m.ts) AS lead_ts,
       CONCAT(u.user_group_id, ' ', LEAD(u.user_group_id) OVER(PARTITION BY thread_ts ORDER BY m.ts)) AS team_communicated,
       LEAD(to_timestamp(CAST(m.ts AS DOUBLE PRECISION))) OVER(PARTITION BY thread_ts ORDER BY m.ts) -
       to_timestamp(CAST(m.ts AS DOUBLE PRECISION)) AS reply_time

FROM webeet_slack.message m
   JOIN webeet_slack.user_group_user u ON m.user_id = u.user_id
   JOIN webeet_slack.channel_member cm ON cm.user_id = m.user_id
   JOIN webeet_slack.channel c ON c.id = cm.channel_id
WHERE thread_ts is not null AND (m.subtype is null OR m.subtype = 'thread_broadcast') AND (c.name is not null
   AND c.name NOT IN ('memes','almuni','Networking','announncements'))
"""
df3 = pd.read_sql_query(query3, engine)
df3.head(2)

,sender_team,thread_ts,sender_user_id,name,ts_new,replier_user_id,replier_team,lead_ts,team_communicated,reply_time
0,S081ZR5L7LG,1.730718e+09,U07S3E41A49,psycholab,2024-11-04 11:06:02.193819+00:00,U07S3E41A49,S07SNMR85FF,2024-11-04 11:06:02.193819+00:00,S081ZR5L7LG S07SNMR85FF,0 days
1,S07SNMR85FF,1.730718e+09,U07S3E41A49,psycholab,2024-11-04 11:06:02.193819+00:00,U07S3E41A49,S081ZR5L7LG,2024-11-04 11:06:02.193819+00:00,S07SNMR85FF S081ZR5L7LG,0 days


In [16]:
# exclude rows where replies_team is none.
df3_1 = df3[df3['replier_team'].notna()]

In [17]:
query4 = """
SELECT id, handle
FROM webeet_slack.user_group
"""
df4 = pd.read_sql_query(query4, engine)
df4.head()

,id,handle
0,S0814D0BR1U,security-team
1,S0814CXDVFG,data-team
2,S081ZR5L7LG,engineering-team
3,S07SNMR85FF,leads


In [18]:
# calculate the average time and the number of threads involved per group communication.
stat_teams = df3_1.groupby('team_communicated').agg(thread_count =('thread_ts','nunique'),
                                                    avg_reply_time = ('reply_time','mean')).reset_index()
stat_teams.head(1)

,team_communicated,thread_count,avg_reply_time
0,S07SNMR85FF S07SNMR85FF,517,0 days 00:15:29.526714338


In [20]:
# define a function that splits the team_communicated field in to sender and replier.
def split_user_groups(input_string):
    # Split the input string by spaces
    parts = input_string.strip().split()

    # Check if we have at least two parts
    if len(parts) >= 2:
        sender = parts[0]
        replier = parts[1]
        return pd.Series([sender, replier])
    else:
        # Handle case with insufficient parts
        return pd.Series([parts[0] if parts else None, None])

# Apply the function to create new columns
stat_teams[['sender_team', 'replier_team']] = stat_teams['team_communicated'].apply(split_user_groups)

# Now the DataFrame has two new columns: 'sender' and 'replier'
teams_communication = stat_teams.drop(columns='team_communicated')
display(teams_communication.head(1))

,thread_count,avg_reply_time,sender_team,replier_team
0,517,0 days 00:15:29.526714338,S07SNMR85FF,S07SNMR85FF


In [22]:
# re_order the column names.
new_column = ['sender_team','replier_team','thread_count','avg_reply_time']
teams_communication = teams_communication[new_column]
teams_communication = teams_communication.sort_values(by='avg_reply_time', ascending=False, ignore_index=True)
teams_communication.head(1)

,sender_team,replier_team,thread_count,avg_reply_time
0,S081ZR5L7LG,S0814D0BR1U,35,0 days 16:33:30.694561750


In [23]:
# insert the group name of the sender team and replied name
# Create a dictionary mapping from ID to team name for faster lookup
id_to_team = dict(zip(df4['id'], df4['handle']))

# Add team names to the first DataFrame
teams_communication['sender_team_name'] = teams_communication['sender_team'].map(id_to_team)
teams_communication['replier_team_name'] = teams_communication['replier_team'].map(id_to_team)
teams_communication.head(1)

,sender_team,replier_team,thread_count,avg_reply_time,sender_team_name,replier_team_name
0,S081ZR5L7LG,S0814D0BR1U,35,0 days 16:33:30.694561750,engineering-team,security-team


In [26]:
# convert the reply time into hours.
def interval_to_hours(interval_str):
    if pd.isna(interval_str):
        return None

    # Parse the time components
    parts = str(interval_str).split()
    days = int(parts[0])
    # Split time part (handle microseconds properly)
    time_parts = parts[2].split('.')
    h, m, s = map(int, time_parts[0].split(':'))

    # Add microseconds if present
    seconds = s
    if len(time_parts) > 1:
        seconds += float('0.' + time_parts[1])

    # Calculate total hours
    return days * 24 + h + m/60 + s/3600

# Apply to your DataFrame
teams_communication['avg_reply_hours'] = teams_communication['avg_reply_time'].apply(interval_to_hours)
teams_communication.head(1)

,sender_team,replier_team,thread_count,avg_reply_time,sender_team_name,replier_team_name,avg_reply_hours
0,S081ZR5L7LG,S0814D0BR1U,35,0 days 16:33:30.694561750,engineering-team,security-team,16.558333


In [34]:
# provide the final table
select_columns_final = ['sender_team_name','replier_team_name','thread_count','avg_reply_hours']
teams_communication_final = teams_communication[select_columns_final]
teams_communication_final.loc[:,'aver_reply_minutes'] = teams_communication_final['avg_reply_hours'] * 60
teams_communication_final.sort_values(by='avg_reply_hours', ascending=False, ignore_index=True).head(3)

<ipython-input-34-1339a2a81f93>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_communication_final.loc[:,'aver_reply_minutes'] = teams_communication_final['avg_reply_hours'] * 60


,sender_team_name,replier_team_name,thread_count,avg_reply_hours,aver_reply_minutes
0,engineering-team,security-team,35,16.558333,993.500000
1,data-team,security-team,22,12.820833,769.250000
2,data-team,engineering-team,39,8.194444,491.666667


In [31]:
# save the df into csv
teams_communication_final.to_csv('teams_communication.csv', index=False)